# [cknowledge.org](http://cknowledge.org): Community-driven benchmarking and optimization of computing systems: from classical to quantum

[Quantum Computing](https://github.com/ctuning/ck-quantum/wiki)
* [CK-QISKit](https://github.com/ctuning/ck-qiskit) (IBM)
* [CK-Rigetti](https://github.com/ctuning/ck-rigetti)
* [CK-ProjectQ](https://github.com/ctuning/ck-projectq)

[Artificial Intelligence and Machine Learning](http://cknowledge.org/ai)
* [Reproducible Quality-Efficient Systems Tournaments](http://cknowledge.org/request) ([ReQuEST initiative](http://cknowledge.org/request.html#organizers))
* [AI artifacts](http://cknowledge.org/ai-artifacts) (cTuning foundation)
* [Android app](https://play.google.com/store/apps/details?id=openscience.crowdsource.video.experiments) (dividiti)
* [Desktop app](https://github.com/dividiti/ck-crowdsource-dnn-optimization) (dividiti)
* [CK-Caffe](https://github.com/dividiti/ck-caffe) (Berkeley)
* [CK-Caffe2](https://github.com/ctuning/ck-caffe2) (Facebook)
* [CK-CNTK](https://github.com/ctuning/ck-cntk) (Microsoft)
* [CK-KaNN](https://github.com/dividiti/ck-kann) (Kalray)
* [CK-MVNC](https://github.com/ctuning/ck-mvnc) (Movidius / Intel)
* [CK-MXNet](https://github.com/ctuning/ck-mxnet) (Apache)
* [CK-NNTest](https://github.com/ctuning/ck-nntest) (cTuning foundation)
* [CK-TensorFlow](https://github.com/ctuning/ck-tensorflow) (Google)
* [CK-TensorRT](https://github.com/dividiti/ck-tensorrt) (NVIDIA)
* etc.

# VQE demo on Rigetti QVM

## Table of Contents

1. [Overview](#overview)
1. [Experimental platform](#platform)
1. [Experimental data](#data) [for developers]
1. [Data wrangling code](#code) [for developers]

<a id="overview"></a>
## Overview

This Jupyter Notebook studies the performance of [Variational-Quantum-Eigensolver](http://grove-docs.readthedocs.io/en/latest/vqe.html) (VQE) on Rigetti's [Quantum Virtual Machine](http://pyquil.readthedocs.io/en/latest/qvm.html) (QVM).

<a id="platform"></a>
## Platform

_TODO_

<a id="data"></a>
## Get the experimental data

The experimental data can be downloaded and registered with CK as described below.

```
$ wget https://www.dropbox.com/s/<...>/ck-rigetti-vqe-demo.zip
$ ck add repo --zip=ck-rigetti-vqe-demo.zip
```

In [ ]:
repo_uoa = 'ck-rigetti-vqe-demo'
!ck list $repo_uoa:experiment:* --print_full | sort

<a id="code"></a>
## Data wrangling code

**NB:** Please ignore this section if you are not interested in re-running or modifying this notebook.

### Includes

#### Standard

In [ ]:
import os
import sys
import json
import re

#### Scientific

If some of the scientific packages are missing, please install them using:
```
# pip install jupyter pandas numpy matplotlib
```

In [ ]:
import IPython as ip
import pandas as pd
import numpy as np
import matplotlib as mp
import seaborn as sb

In [ ]:
print ('IPython version: %s' % ip.__version__)
print ('Pandas version: %s' % pd.__version__)
print ('NumPy version: %s' % np.__version__)
print ('Matplotlib version: %s' % mp.__version__)
print ('Seaborn version: %s' % sb.__version__)

In [ ]:
from IPython.display import Image, display
def display_in_full(df):
    pd.options.display.max_columns = len(df.columns)
    pd.options.display.max_rows = len(df.index)
    display(df)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline

In [ ]:
default_colormap = cm.autumn
default_fontsize = 16
default_barwidth = 0.8
default_figwidth = 24
default_figheight = 3
default_figdpi = 200
default_figsize = [default_figwidth, default_figheight]

In [ ]:
if mp.__version__[0]=='2': mp.style.use('classic')
mp.rcParams['figure.max_open_warning'] = 200
mp.rcParams['figure.dpi'] = default_figdpi
mp.rcParams['font.size'] = default_fontsize
mp.rcParams['legend.fontsize'] = 'medium'

#### Collective Knowledge

If CK is not installed, please install it using:
```
# pip install ck
```

In [ ]:
import ck.kernel as ck
print ('CK version: %s' % ck.__version__)

### Access experimental data

In [ ]:
def get_experimental_results(repo_uoa, tags='explore-method-iteration-sample', module_uoa='experiment'):
    r = ck.access({'action':'search', 'repo_uoa':repo_uoa, 'module_uoa':module_uoa, 'tags':tags})
    if r['return']>0:
        print('Error: %s' % r['error'])
        exit(1)
    experiments = r['lst']
    
    dfs = []
    for experiment in experiments:
        skip = False
        data_uoa = experiment['data_uoa']
        r = ck.access({'action':'list_points', 'repo_uoa':repo_uoa, 'module_uoa':module_uoa, 'data_uoa':data_uoa})
        if r['return']>0:
            print('Error: %s' % r['error'])
            exit(1)
        tags = r['dict']['tags']
        
        platform_tags = [ tag for tag in tags if tag in ['rigetti-pyquil'] ]
        if len(platform_tags)!=1: skip = True
        
        program_tags = [ tag for tag in tags if tag in ['vqe-helium-example'] ]
        if len(program_tags)!=1: skip = True
        
        if skip:
            print('[Warning] Bad library tags. Skipping experiment with tags:')
            print(tags)
            continue
        # For each point.    
        for point in r['points']:
            point_file_path = os.path.join(r['path'], 'ckp-%s.0001.json' % point)
            with open(point_file_path) as point_file:
                point_data_raw = json.load(point_file)
            characteristics_list = point_data_raw['characteristics_list']
            num_repetitions = len(characteristics_list)
            data = [
                {
                    # features
                    'platform': platform_tags[0],
                    # choices
                    'minimizer_method': characteristics['run'].get('vqe_input', {}).get('minimizer_method','n/a'),
                    'max_iterations': characteristics['run'].get('vqe_input', {}).get('max_iterations','n/a'),
                    'sample_number': characteristics['run'].get('vqe_input', {}).get('sample_number','n/a'),
                    # statistical repetition
                    'repetition_id': repetition_id,
                    # runtime characteristics
                    'report': characteristics['run'].get('report', {}),
                    'vqe_output': characteristics['run'].get('vqe_output', {}),
                }
                for (repetition_id, characteristics) in zip(range(num_repetitions), characteristics_list)
            ]
            for datum in data:
                datum['success'] = datum.get('vqe_output',{}).get('success',False)
                datum['nfev'] = np.int64(datum.get('vqe_output',{}).get('nfev',-1))
                datum['nit'] = np.int64(datum.get('vqe_output',{}).get('nit',-1))
                datum['fun'] = np.float64(datum.get('vqe_output',{}).get('fun',0))
                datum['total_seconds'] = np.float64(datum.get('report',{}).get('total_seconds',0))
                datum['total_q_seconds'] = np.float64(datum.get('report',{}).get('total_q_seconds',0))
            index = [
                'platform', 'minimizer_method', 'max_iterations', 'sample_number', 'repetition_id'
            ]
            # Construct a DataFrame.
            df = pd.DataFrame(data)
            df = df.set_index(index)
            # Append to the list of similarly constructed DataFrames.
            dfs.append(df)
    if dfs:
        # Concatenate all thus constructed DataFrames (i.e. stack on top of each other).
        result = pd.concat(dfs)
        result.sort_index(ascending=True, inplace=True)
    else:
        # Construct a dummy DataFrame the success status of which can be safely checked.
        result = pd.DataFrame(columns=['success'])
    return result

### Plot experimental data

_TODO_

<a id="analysis"></a>
## Analysis

In [ ]:
df = get_experimental_results(repo_uoa=repo_uoa)
display_in_full(df)